In [2]:
import requests
from bs4 import BeautifulSoup

DOC_TYPE_CEDULA_CHILENA = "CEDULA"
DOC_TYPE_CEDULA_EXTRANJEROS = "CEDULA_EXT"
DOC_TYPE_PASAPORTE_PG = "PASAPORTE_PG"
DOC_TYPE_PASAPORTE_DIPLOMATICO = "PASAPORTE_DIPLOMATICO"
DOC_TYPE_PASAPORTE_OFICIAL = "PASAPORTE_OFICIAL"

def validar_documento(run, docNumber, docType):
    session = requests.Session()
    session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'})
    session.verify = False

    # init session
    response = session.get('https://portal.sidiv.registrocivil.cl/usuarios-portal/pages/DocumentRequestStatus.xhtml')
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'lxml')
    captcha_url = 'https://portal.sidiv.registrocivil.cl'+soup.select('#form\:captchaPanel img')[0].get('src')
    print('captcha_url', captcha_url)
    view_state = soup.find(id='javax.faces.ViewState').get('value')
    print('view_state', view_state)

    
    # solve captcha
    response = requests.post('https://captcha-solvers.herokuapp.com/myfacescaptcha', data={
        'image_url': captcha_url
    })
    response.raise_for_status()
    captcha_text = response.json()['text']
    print('captcha_text', captcha_text)

    # request status
    formdata = {
        'form': 'form',
        'form:captchaUrl': 'initial',
        'form:run': run,
        'form:selectDocType': docType,
        'form:docNumber': docNumber,
        'form:inputCaptcha': captcha_text,
        'form:buttonHidden': '',
        'javax.faces.ViewState': view_state
    }
    response = session.post('https://portal.sidiv.registrocivil.cl/usuarios-portal/pages/DocumentRequestStatus.xhtml', data=formdata)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'lxml')

    status = soup.find(class_='setWidthOfSecondColumn').get_text(strip=True)
    if not status: raise Exception("DOCUMENT DOES NOT MATCH!")
    return status

In [7]:
status = validar_documento('15011651-1', '521264669', DOC_TYPE_CEDULA_CHILENA)
print('status', status.upper())

captcha_url https://portal.sidiv.registrocivil.cl/usuarios-portal/faces/myFacesExtensionResource/org.apache.myfaces.custom.captcha.CAPTCHARenderer/15923103/;jsessionid=4019345E1B76224806A63A36C7836197.tomcat5?captchaSessionKeyName=mySessionKeyName
view_state -6603055237055888119:-6875277701094112449
captcha_text SNHn8
status VIGENTE
